In [13]:
import pandas as pd
from error_config import error_map

In [14]:
filename_refmatters = '/home/sanjana/factual_evaluation_source_based/datasets/sota_annotations/FacEval.csv'
filename_dialsum = '/home/sanjana/factual_evaluation_source_based/datasets/sota_annotations/RefMatters_DialogSum_test.csv'
filename_samsum = '/home/sanjana/factual_evaluation_source_based/datasets/sota_annotations/RefMatters_SAMSum_test.csv'
df_refmatters = pd.read_csv(filename_refmatters)
df_dialsum = pd.read_csv(filename_dialsum)
df_samsum = pd.read_csv(filename_samsum)



In [15]:
df_refmatters

,Unnamed: 0,DocID,Dialogue,Model,Summary,SubObjE,ProE,NegE,ParE,HalE,OtherE,w/ Error
0,0,13809941,Thelma: i dont have anything to wear\nLouisa: ...,human_ref,Louisa will lend Thelma her red velvet dress.,no,no,no,no,no,no,no
1,1,13809941,Thelma: i dont have anything to wear\nLouisa: ...,bart_large,Thelma doesn't have anything to wear. Louisa w...,no,no,no,no,no,no,no
2,2,13809941,Thelma: i dont have anything to wear\nLouisa: ...,mv-bart_large,Thelma doesn't have anything to wear. Louisa w...,no,no,no,no,no,no,no
3,3,13809941,Thelma: i dont have anything to wear\nLouisa: ...,co-ref bart large,Thelma doesn't have anything to wear. Louisa w...,no,no,no,no,no,no,no
4,4,13809941,Thelma: i dont have anything to wear\nLouisa: ...,condigsum bart large,Louisa will bring Thelma her red velvet dress.,no,no,no,no,no,no,no
...,...,...,...,...,...,...,...,...,...,...,...,...
745,745,13829937,Nathan: <file_video>\nAaron: OMG!!! \nAaron: \...,human_ref,Nathan and Aaron are discussing a video which ...,no,no,no,no,no,no,no
746,746,13829937,Nathan: <file_video>\nAaron: OMG!!! \nAaron: \...,bart_large,Dan sent Aaron a video of a cat taking a dump.,yes,no,no,no,no,no,yes
747,747,13829937,Nathan: <file_video>\nAaron: OMG!!! \nAaron: \...,mv-bart_large,Nathan and Dan sent Aaron a video of a woman h...,yes,no,no,yes,no,no,yes
748,748,13829937,Nathan: <file_video>\nAaron: OMG!!! \nAaron: \...,co-ref bart large,Nathan and Aaron are laughing at a video of a ...,no,no,no,yes,no,no,yes


In [16]:
error_types = list(set(error_map.values()))
# error_types

In [36]:
from math import nan
def format_data(df, error_types, origin ):
    df_aggregate = {}
    df_aggregate['DocID'] = []
    df_aggregate['Dialogue'] = []
    df_aggregate['Model'] = []
    df_aggregate['Summary'] = []
    df_aggregate['w/ Error'] = []
    for err_type in error_types:
            df_aggregate[err_type] = []
            df_aggregate[f'{err_type}_text'] = []
    

    for idx, row in df.iterrows():
        df_aggregate['DocID'] += [row['DocID']]
        df_aggregate['Dialogue'] += [row['Dialogue']]
        df_aggregate['Model'] += [row['Model']]
        df_aggregate['Summary'] += [row['Summary']]
        w_error = 0
    
        for err_type in error_types:
            err_val = nan
            err_txt = nan
            if err_type in row:
                err_val = row[err_type]
                if type(err_val) is str:
                    err_val = 1 if err_val == 'yes' else 0
                
                err_txt = row[f'{err_type}_text'] if f'{err_type}_text' in row else nan
                if err_val == 1:
                    w_error = 1
                
            df_aggregate[err_type].append(err_val)
            df_aggregate[f'{err_type}_text'].append(err_txt)
        
        df_aggregate['w/ Error'] += [w_error] 

    df_aggregate['origin'] = [origin] * len(df_aggregate['DocID'])
    return pd.DataFrame(df_aggregate)
   

In [37]:
df_refmatters = format_data(df_refmatters, error_types, 'FacEval')
df_dialsum = format_data(df_dialsum, error_types, 'RefMatters_DialogSu')
df_samsum = format_data(df_samsum, error_types, 'RefMatters_SAMSum')

In [38]:
df_dialogue_aggr = pd.concat([df_refmatters, df_dialsum, df_samsum])

In [39]:
len(df_dialogue_aggr)

1550

In [40]:
list(set(df_dialogue_aggr['w/ Error'].values))

[0, 1]

In [41]:
df_dialogue_aggr

,DocID,Dialogue,Model,Summary,w/ Error,CorefE,CorefE_text,CircE,CircE_text,OutE,...,GramE_text,PredE,PredE_text,SubjObjE,SubjObjE_text,OtherE,OtherE_text,LinkE,LinkE_text,origin
0,13809941,Thelma: i dont have anything to wear\nLouisa: ...,human_ref,Louisa will lend Thelma her red velvet dress.,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,FacEval
1,13809941,Thelma: i dont have anything to wear\nLouisa: ...,bart_large,Thelma doesn't have anything to wear. Louisa w...,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,FacEval
2,13809941,Thelma: i dont have anything to wear\nLouisa: ...,mv-bart_large,Thelma doesn't have anything to wear. Louisa w...,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,FacEval
3,13809941,Thelma: i dont have anything to wear\nLouisa: ...,co-ref bart large,Thelma doesn't have anything to wear. Louisa w...,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,FacEval
4,13809941,Thelma: i dont have anything to wear\nLouisa: ...,condigsum bart large,Louisa will bring Thelma her red velvet dress.,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,FacEval
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,13864835,Tony: <photo_file>\nAmy: Sweet little cat <3\n...,CODS,Amy sends Lucas a photo of a cat.,1,0.0,NaN,0.0,NaN,0.0,...,NaN,0.0,NaN,1.0,Amy,NaN,NaN,0.0,NaN,RefMatters_SAMSum
396,13727934,"Patricia: Hey, could you give me a call in an ...",BART,George will call Patricia in an hour.,0,0.0,NaN,0.0,NaN,0.0,...,NaN,0.0,NaN,0.0,NaN,NaN,NaN,0.0,NaN,RefMatters_SAMSum
397,13727934,"Patricia: Hey, could you give me a call in an ...",UniLM,George will call Patricia in an hour or so bec...,1,1.0,she,0.0,NaN,0.0,...,NaN,0.0,NaN,0.0,NaN,NaN,NaN,0.0,NaN,RefMatters_SAMSum
398,13727934,"Patricia: Hey, could you give me a call in an ...",MV-BART,Patricia wants George to call her in an hour.,0,0.0,NaN,0.0,NaN,0.0,...,NaN,0.0,NaN,0.0,NaN,NaN,NaN,0.0,NaN,RefMatters_SAMSum


In [42]:
df_dialogue_aggr.to_csv('/home/sanjana/factual_evaluation_source_based/datasets/sota_annotations/dialogue_aggrefact.csv')

